Я тут хочу написать общие вопросы по заданию, и в самом задании, пишу в комментариях тоже.
Первый, наверное, по поводу оформления задания, есть ли какие-то отличительные черты как оформляю его я и как бы сделал это человек знающий:) Что меня здесь сдает, что я новичек, я не имею ввиду качество кода, он так точно сдает :) 

Второе, огромный вопрос по работе с признаками. Ты мне его задавал, чтобы я с ними делал, как их обрабатывал, и я так понимаю сейчас на практике, что ответ я знаю процентов на 30. Какой-то может общий план: как определить какие признаки важны, а какие не очень, а какие сразу отбросить. В задании я удаляю признаки из vendors, делаю это примерно по принципу нравится/не нравится. Наверное есть более научный подход:)

Что делать с признаками которые оставили, дополни, пожалуйста: числовые признаки - я проверяю пропущеные поля, заполняю их средним значением, либо нулем. Вроде где-то на курсе говорили, что xboost (ты говорил что есть уже что-то лучше, где можно почитать?) - сам как-то работает с пропущеными признаками, или это мне показалось. Масштабирую признаки при необходимости
Категориальные признаки - кодирую LabelEncoder - просто меняет значение на ранг, либо что вроде логично, но не применимо когда категорий очень много - OneHotEncoder. Наверное есть еще варианты
Есть ли какой-то принцип по добавлению новых признаков для улучшения модели?





In [511]:
import numpy as np
import pandas as pd
from collections import  Counter
import xgboost as xgb
from sklearn import model_selection, metrics, ensemble
from sklearn.feature_extraction import DictVectorizer as DV

### Загружаю данные

In [512]:
train_customers = pd.read_csv('train_customers.csv', header = 0, sep = ',')
test_customers = pd.read_csv('test_customers.csv', header = 0, sep = ',')
orders = pd.read_csv('orders.csv', header = 0, sep = ',')
vendors = pd.read_csv('vendors.csv', header = 0, sep = ',')
train_locations = pd.read_csv('train_locations.csv', header = 0, sep = ',')
test_locations = pd.read_csv('test_locations.csv', header = 0, sep = ',')
SampleSubmission = pd.read_csv('SampleSubmission.csv', header = 0, sep = ',')

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (15,16,18,19,20) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [513]:
train_customers.head()

,akeed_customer_id,gender,dob,status,verified,language,created_at,updated_at
0,TCHWPBT,Male,NaN,1,1,EN,2018-02-07 19:16:23,2018-02-07 19:16:23
1,ZGFSYCZ,Male,NaN,1,1,EN,2018-02-09 12:04:42,2018-02-09 12:04:41
2,S2ALZFL,Male,NaN,0,1,EN,2018-03-14 18:31:43,2018-03-14 18:31:42
3,952DBJQ,Male,NaN,1,1,EN,2018-03-15 19:47:07,2018-03-15 19:47:07
4,1IX6FXS,Male,NaN,1,1,EN,2018-03-15 19:57:01,2018-03-15 19:57:01


In [514]:
test_customers.head()

,akeed_customer_id,gender,dob,status,verified,language,created_at,updated_at
0,ICE2DJP,Male,NaN,1,1,EN,2018-02-07 16:45:36,2018-02-07 16:45:36
1,FWNUI71,Male,NaN,1,1,EN,2018-03-22 20:11:47,2018-03-22 20:11:47
2,LRX7BCH,Male,NaN,1,1,EN,2018-04-17 20:01:15,2018-04-17 20:01:15
3,D96DHMD,Male,NaN,1,1,EN,2018-04-29 22:35:52,2018-04-29 22:35:52
4,88Q8Y5V,Male,1997.0,1,1,EN,2018-05-05 12:38:09,2018-05-05 12:38:09


In [515]:
orders.head()

,akeed_order_id,customer_id,item_count,grand_total,payment_mode,promo_code,vendor_discount_amount,promo_code_discount_percentage,is_favorite,is_rated,...,driver_accepted_time,ready_for_pickup_time,picked_up_time,delivered_time,delivery_date,vendor_id,created_at,LOCATION_NUMBER,LOCATION_TYPE,CID X LOC_NUM X VENDOR
0,163238.0,92PEE24,1.0,7.6,2,NaN,0.0,NaN,NaN,No,...,NaN,NaN,NaN,NaN,2019-07-31 05:30:00,105,2019-08-01 05:30:16,0,NaN,92PEE24 X 0 X 105
1,163240.0,QS68UD8,1.0,8.7,1,NaN,0.0,NaN,NaN,No,...,NaN,NaN,NaN,NaN,2019-07-31 05:30:00,294,2019-08-01 05:31:10,0,Work,QS68UD8 X 0 X 294
2,163241.0,MB7VY5F,2.0,14.4,1,NaN,0.0,NaN,NaN,No,...,NaN,NaN,NaN,NaN,2019-07-31 05:30:00,83,2019-08-01 05:31:33,0,NaN,MB7VY5F X 0 X 83
3,163244.0,KDJ951Y,1.0,7.1,1,NaN,0.0,NaN,NaN,No,...,NaN,NaN,NaN,NaN,2019-07-31 05:30:00,90,2019-08-01 05:34:54,0,Home,KDJ951Y X 0 X 90
4,163245.0,BAL0RVT,4.0,27.2,1,NaN,0.0,NaN,NaN,No,...,NaN,NaN,NaN,NaN,2019-07-31 05:30:00,83,2019-08-01 05:35:51,0,Work,BAL0RVT X 0 X 83


In [516]:
vendors.head()

,id,authentication_id,latitude,longitude,vendor_category_en,vendor_category_id,delivery_charge,serving_distance,is_open,OpeningTime,...,open_close_flags,vendor_tag,vendor_tag_name,one_click_vendor,country_id,city_id,created_at,updated_at,device_type,display_orders
0,4,118597.0,-0.588596,0.754434,Restaurants,2.0,0.0,6.0,1.0,11:00AM-11:30PM,...,1.0,"2,4,5,8,91,22,12,24,16,23","Arabic,Breakfast,Burgers,Desserts,Free Deliver...",Y,1.0,1.0,2018-01-30 14:42:04,2020-04-07 15:12:43,3,1
1,13,118608.0,-0.471654,0.744470,Restaurants,2.0,0.7,5.0,1.0,08:30AM-10:30PM,...,1.0,"4,41,51,34,27,15,24,16,28","Breakfast,Cakes,Crepes,Italian,Pasta,Pizzas,Sa...",Y,1.0,1.0,2018-05-03 12:32:06,2020-04-05 20:46:03,3,1
2,20,118616.0,-0.407527,0.643681,Restaurants,2.0,0.0,8.0,1.0,08:00AM-10:45PM,...,1.0,"4,8,91,10","Breakfast,Desserts,Free Delivery,Indian",Y,1.0,1.0,2018-05-04 22:28:22,2020-04-07 16:35:55,3,1
3,23,118619.0,-0.585385,0.753811,Restaurants,2.0,0.0,5.0,1.0,10:59AM-10:30PM,...,1.0,"5,8,30,24","Burgers,Desserts,Fries,Salads",Y,1.0,1.0,2018-05-06 19:20:48,2020-04-02 00:56:17,3,1
4,28,118624.0,0.480602,0.552850,Restaurants,2.0,0.7,15.0,1.0,11:00AM-11:45PM,...,1.0,5,Burgers,Y,1.0,1.0,2018-05-17 22:12:38,2020-04-05 15:57:41,3,1


In [517]:
train_locations.head()

,customer_id,location_number,location_type,latitude,longitude
0,02SFNJH,0,NaN,1.682392,-78.789737
1,02SFNJH,1,NaN,1.679137,0.766823
2,02SFNJH,2,NaN,-0.498648,0.661241
3,RU43CXC,0,Home,0.100853,0.438165
4,BDFBPRD,0,NaN,2.523125,0.733464


In [518]:
print (SampleSubmission.head())
print (SampleSubmission.shape)

  CID X LOC_NUM X VENDOR  target
0      Z59FTQD X 0 X 243       0
1      0JP29SK X 0 X 243       0
2      0JP29SK X 1 X 243       0
3      0JP29SK X 2 X 243       0
4      0JP29SK X 3 X 243       0
(1672000, 2)


In [519]:
# смотрю количество клиентов в train, test
print (train_customers.shape)
print (test_customers.shape)

(34674, 8)
(9768, 8)


### Обработка признаков

#### train_customers, test_customers

In [521]:
# смотрю сколько в столбцах пропущенных значений, дата рождения пропущена почту во всех столбцах, думаю есть
# смысл ее удалить. Вопрос: есть ли какой то стандартный, средний процент, сколько должно быть данных в признаке,
# чтобы его оставить/удалить?
# Выглядит так же лишним один из признаков created_at и updated_at, пока удалю оба, не знаю как их обработать
train_customers.isnull().sum(axis = 0)

akeed_customer_id        0
gender               12154
dob                  31628
status                   0
verified                 0
language             13575
created_at               0
updated_at               0
dtype: int64

In [522]:
# удаляю
train_customers.drop(['dob', 'created_at', 'updated_at'], axis = 1, inplace = True)
test_customers.drop(['dob', 'created_at', 'updated_at'], axis = 1, inplace = True)
train_customers.shape

(34674, 5)

In [523]:
#хочу посмотреть как распеделяется значения признаков. Вопрос: наверное это можно сделать покрасивее?
#Вижу что здесь надо поменять название пола, так как его вносили поразному. Странная ситуация с языком арабской стране
# большая часть пользователей выбирают английский, видимо те кто не выбирают - арабский, значит большая часть 
# пользователей туристы/экспаты, может это важно :)
print ('gender_train:', Counter(train_customers.gender))
print ('gender_test:', Counter(test_customers.gender))
print ('status_train:', Counter(train_customers.status))
print ('status_test:', Counter(test_customers.status))
print ('verified_train:', Counter(train_customers.verified))
print ('verified_test:', Counter(test_customers.verified))
print ('language_train:', Counter(train_customers.language))
print ('language_test:', Counter(test_customers.language))


gender_train: Counter({'Male': 17815, nan: 12154, 'male': 2914, 'Female': 1761, 'Female  ': 13, 'Male  ': 9, 'Female   ': 2, 'Female ': 2, '?????': 2, 'Female    ': 1, '  ': 1})
gender_test: Counter({'Male': 5021, nan: 3447, 'male': 777, 'Female': 514, 'Female  ': 6, 'Female ': 2, 'Male  ': 1})
status_train: Counter({1: 34639, 0: 35})
status_test: Counter({1: 9758, 0: 10})
verified_train: Counter({1: 33167, 0: 1507})
verified_test: Counter({1: 9334, 0: 434})
language_train: Counter({'EN': 21099, nan: 13575})
language_test: Counter({'EN': 5928, nan: 3840})


Хочу поменять пол: женский - 2, мужской - 1, неопределен - 0. Тут есть пару вопросов, первый по реализации, вроде
не очень красиво, но с другой стороны так заполнены данные. Хотя, наверное, то как я меняю данные, должно быть применимо для новых клиентов, для которых в будущем будет делаться предсказание. Второй: имеет ли значение, как кодировать, так как
я сделал, или лучше применить OneHotEncoder ? 

In [524]:
train_customers = train_customers.replace(to_replace=['Female', 'Female  ', 'Female   ', 'Female ', 'Female    ',
                                                     'Male', 'male', 'Male  ', '?????', '  '], 
                                          value=[2, 2, 2, 2, 2, 1, 1, 1, 0, 0])
train_customers['gender'].fillna(0, inplace=True)

test_customers = test_customers.replace(to_replace=['Female', 'Female  ', 'Female   ', 'Female ', 'Female    ',
                                                     'Male', 'male', 'Male  ', '?????', '  '], 
                                          value=[2, 2, 2, 2, 2, 1, 1, 1, 0, 0])
test_customers['gender'].fillna(0, inplace=True)

In [525]:
Counter(test_customers['gender'])

Counter({1.0: 5799, 2.0: 522, 0.0: 3447})

In [526]:
# меняю language
train_customers = train_customers.replace(to_replace='EN', value=1)
train_customers['language'].fillna(0, inplace=True)

test_customers = test_customers.replace(to_replace='EN', value=1)
test_customers['language'].fillna(0, inplace=True)

In [527]:
Counter(test_customers['language'])

Counter({1.0: 5928, 0.0: 3840})

In [528]:
train_customers.head()

,akeed_customer_id,gender,status,verified,language
0,TCHWPBT,1.0,1,1,1.0
1,ZGFSYCZ,1.0,1,1,1.0
2,S2ALZFL,1.0,0,1,1.0
3,952DBJQ,1.0,1,1,1.0
4,1IX6FXS,1.0,1,1,1.0


In [529]:
# хочу удалить повторяющихся клиентов
train_customers.drop_duplicates('akeed_customer_id', inplace = True)
test_customers.drop_duplicates('akeed_customer_id', inplace = True)

In [530]:
# смотрю новое количество клиентов, стало немного меньше
print (train_customers.shape)
print (test_customers.shape)

(34523, 5)
(9753, 5)


#### vendors

Дальше я перебираю все признаки в vendors на предмет поиска тех, что можно удалить, не знаю как это правильно
оформить, я просто смотрю каждый признак. 

authentication_id - удаляю, так как он не несет смысловой нагрузки, OpeningTime2 - выглядит непонятным и лишним, vendor_tag_name- дублирует vendor_tag, commission - удаляю, все значения или 0 или незаполнены


OpeningTime - удаляю, не знаю как обработать, sunday_from_time1 и т.д - не знаю как это обработать, поэтому удаляю все подобные признаки, created_at - дублируется с updated_at - удаляю пока оба, не знаю как их обработать, primary_tags - удаляю, не знаю как это обработать, vendor_tag - удаляю, не знаю как это обработать, updated_at - удаляю, не знаю как это обработать


is_akeed_delivering - все значения одинаковые, open_close_flags - значение везде одинаковое, one_click_vendor -все значения одинаковые,  country_id - все значения одинаковые, city_id - все значения одинаковы, display_orders - все значения одинаковые.

In [531]:
vendors.columns

Index(['id', 'authentication_id', 'latitude', 'longitude',
       'vendor_category_en', 'vendor_category_id', 'delivery_charge',
       'serving_distance', 'is_open', 'OpeningTime', 'OpeningTime2',
       'prepration_time', 'commission', 'is_akeed_delivering',
       'discount_percentage', 'status', 'verified', 'rank', 'language',
       'vendor_rating', 'sunday_from_time1', 'sunday_to_time1',
       'sunday_from_time2', 'sunday_to_time2', 'monday_from_time1',
       'monday_to_time1', 'monday_from_time2', 'monday_to_time2',
       'tuesday_from_time1', 'tuesday_to_time1', 'tuesday_from_time2',
       'tuesday_to_time2', 'wednesday_from_time1', 'wednesday_to_time1',
       'wednesday_from_time2', 'wednesday_to_time2', 'thursday_from_time1',
       'thursday_to_time1', 'thursday_from_time2', 'thursday_to_time2',
       'friday_from_time1', 'friday_to_time1', 'friday_from_time2',
       'friday_to_time2', 'saturday_from_time1', 'saturday_to_time1',
       'saturday_from_time2', 'saturday

In [532]:
delete = ['authentication_id', 'OpeningTime', 'OpeningTime2', 'is_akeed_delivering', 'sunday_from_time1', 'sunday_to_time1',
       'sunday_from_time2', 'sunday_to_time2', 'monday_from_time1',
       'monday_to_time1', 'monday_from_time2', 'monday_to_time2',
       'tuesday_from_time1', 'tuesday_to_time1', 'tuesday_from_time2',
       'tuesday_to_time2', 'wednesday_from_time1', 'wednesday_to_time1',
       'wednesday_from_time2', 'wednesday_to_time2', 'thursday_from_time1',
       'thursday_to_time1', 'thursday_from_time2', 'thursday_to_time2',
       'friday_from_time1', 'friday_to_time1', 'friday_from_time2',
       'friday_to_time2', 'saturday_from_time1', 'saturday_to_time1',
       'saturday_from_time2', 'saturday_to_time2', 'open_close_flags', 'vendor_tag_name', 'one_click_vendor', 
        'country_id', 'city_id', 'created_at', 'updated_at', 'display_orders', 'primary_tags', 'vendor_tag', 
        'updated_at', 'commission']
vendors.drop(vendors[delete], axis = 1, inplace = True)
vendors.shape

(100, 16)

In [533]:
# проверяю на пропущеные значения
vendors.isnull().sum(axis = 0)

id                      0
latitude                0
longitude               0
vendor_category_en      0
vendor_category_id      0
delivery_charge         0
serving_distance        0
is_open                 0
prepration_time         0
discount_percentage     0
status                  0
verified                0
rank                    0
language               15
vendor_rating           0
device_type             0
dtype: int64

In [534]:
vendors.head(100)

,id,latitude,longitude,vendor_category_en,vendor_category_id,delivery_charge,serving_distance,is_open,prepration_time,discount_percentage,status,verified,rank,language,vendor_rating,device_type
0,4,-0.588596,0.754434,Restaurants,2.0,0.0,6.0,1.0,15,0.0,1.0,1,11,EN,4.4,3
1,13,-0.471654,0.744470,Restaurants,2.0,0.7,5.0,1.0,14,0.0,1.0,1,11,EN,4.7,3
2,20,-0.407527,0.643681,Restaurants,2.0,0.0,8.0,1.0,19,0.0,1.0,1,1,EN,4.5,3
3,23,-0.585385,0.753811,Restaurants,2.0,0.0,5.0,1.0,16,0.0,1.0,1,11,EN,4.5,3
4,28,0.480602,0.552850,Restaurants,2.0,0.7,15.0,1.0,10,0.0,1.0,1,11,EN,4.4,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,849,-1.588060,-0.066441,Restaurants,2.0,0.0,10.0,1.0,10,0.0,1.0,1,1,NaN,4.1,3
96,855,2.145206,0.745025,Restaurants,2.0,0.0,8.0,1.0,10,0.0,1.0,1,1,NaN,4.2,3
97,856,0.251469,0.483632,Restaurants,2.0,0.0,7.0,1.0,10,0.0,1.0,1,11,NaN,4.3,3
98,858,0.019817,0.587087,Restaurants,2.0,0.0,3.0,1.0,10,0.0,1.0,1,11,NaN,4.2,3


In [535]:
# обрабатываю признаки в vendor_category_en
vendors = vendors.replace(to_replace=['Restaurants', 'Sweets & Bakes'], value=[1, 0])
# обрабатываю признаки в language
vendors = vendors.replace(to_replace='EN', value=1)
vendors['language'].fillna(0, inplace=True)

vendors.head()

,id,latitude,longitude,vendor_category_en,vendor_category_id,delivery_charge,serving_distance,is_open,prepration_time,discount_percentage,status,verified,rank,language,vendor_rating,device_type
0,4,-0.588596,0.754434,1,2.0,0.0,6.0,1.0,15,0.0,1.0,1,11,1.0,4.4,3
1,13,-0.471654,0.744470,1,2.0,0.7,5.0,1.0,14,0.0,1.0,1,11,1.0,4.7,3
2,20,-0.407527,0.643681,1,2.0,0.0,8.0,1.0,19,0.0,1.0,1,1,1.0,4.5,3
3,23,-0.585385,0.753811,1,2.0,0.0,5.0,1.0,16,0.0,1.0,1,11,1.0,4.5,3
4,28,0.480602,0.552850,1,2.0,0.7,15.0,1.0,10,0.0,1.0,1,11,1.0,4.4,3


#### train_locations, test_locations

In [536]:
train_locations.head(100)

,customer_id,location_number,location_type,latitude,longitude
0,02SFNJH,0,NaN,1.682392,-78.789737
1,02SFNJH,1,NaN,1.679137,0.766823
2,02SFNJH,2,NaN,-0.498648,0.661241
3,RU43CXC,0,Home,0.100853,0.438165
4,BDFBPRD,0,NaN,2.523125,0.733464
...,...,...,...,...,...
95,6YA9PQX,1,Home,-0.504101,0.662351
96,H148LZ3,0,NaN,-0.246295,0.779378
97,6CZCAST,0,Home,0.584490,0.478448
98,6CZCAST,1,Home,-0.434852,0.734251


In [537]:
train_locations.location_type.fillna('NA', inplace=True)
test_locations.location_type.fillna('NA', inplace=True)
train_locations = train_locations.replace(to_replace=['NA', 'Home', 'Other', 'Work'], value=[0, 1, 2, 3])
test_locations = test_locations.replace(to_replace=['NA', 'Home', 'Other', 'Work'], value=[0, 1, 2, 3])

In [538]:
Counter(test_locations.location_type)

Counter({0: 7650, 1: 5504, 2: 1773, 3: 1793})

In [541]:
train_locations.isnull().sum(axis = 0)

customer_id        0
location_number    0
location_type      0
latitude           6
longitude          6
dtype: int64

### Объединяю датафреймы

In [486]:
# переименую колонку с id клиента, чтобы по ней можно было бы объеденить данные
train_customers.rename(columns={'akeed_customer_id':'customer_id'}, inplace = True)
test_customers.rename(columns={'akeed_customer_id':'customer_id'}, inplace = True)

# объединяю по customer_id в один датафрейм клиентов и локации
train_customers = train_customers.merge(train_locations, on = ['customer_id'], how = 'outer')
test_customers = test_customers.merge(test_locations, on = ['customer_id'], how = 'outer')
train_customers.head()

,customer_id,gender,status,verified,language,location_number,location_type,latitude,longitude
0,TCHWPBT,1.0,1.0,1.0,1.0,0,3,-96.407538,-67.197291
1,TCHWPBT,1.0,1.0,1.0,1.0,1,1,-96.645619,-67.169154
2,TCHWPBT,1.0,1.0,1.0,1.0,2,0,-0.128673,-78.575702
3,ZGFSYCZ,1.0,1.0,1.0,1.0,0,1,-0.175558,-78.570161
4,ZGFSYCZ,1.0,1.0,1.0,1.0,1,1,0.191105,-78.613494


In [488]:
print (train_customers.shape)
print (test_customers.shape)

(59503, 9)
(16720, 9)


In [489]:
# в результате объединения клиентов и локаций, я получил таблицу из всех покупок клиентов в разных локациях, поэтому
# количество объектов существенно увеличилось
test_customers.head()

,customer_id,gender,status,verified,language,location_number,location_type,latitude,longitude
0,ICE2DJP,1.0,1.0,1.0,1.0,0,0,-96.407538,-67.197291
1,ICE2DJP,1.0,1.0,1.0,1.0,1,0,0.038654,-78.595477
2,ICE2DJP,1.0,1.0,1.0,1.0,2,2,-95.106078,43.684151
3,ICE2DJP,1.0,1.0,1.0,1.0,3,2,-96.407677,43.557974
4,ICE2DJP,1.0,1.0,1.0,1.0,4,3,204.621247,44.091518


In [490]:
%%time
# для каждого клиента добавляю все строки с ресторанами, в результате получаю число строк в тесте идентичное
# с числом строк из примера SampleSubmission
train=train_customers.assign(key=1).merge(vendors.assign(key=1), on='key').drop('key',axis=1)
test=test_customers.assign(key=1).merge(vendors.assign(key=1), on='key').drop('key',axis=1)

CPU times: user 3.97 s, sys: 5.93 s, total: 9.91 s
Wall time: 14.1 s


In [491]:
print (train.shape)
print (test.shape)
train.head(500)

(5950300, 25)
(1672000, 25)


,customer_id,gender,status_x,verified_x,language_x,location_number,location_type,latitude_x,longitude_x,id,...,serving_distance,is_open,prepration_time,discount_percentage,status_y,verified_y,rank,language_y,vendor_rating,device_type
0,TCHWPBT,1.0,1.0,1.0,1.0,0,3,-96.407538,-67.197291,4,...,6.0,1.0,15,0.0,1.0,1,11,1.0,4.4,3
1,TCHWPBT,1.0,1.0,1.0,1.0,0,3,-96.407538,-67.197291,13,...,5.0,1.0,14,0.0,1.0,1,11,1.0,4.7,3
2,TCHWPBT,1.0,1.0,1.0,1.0,0,3,-96.407538,-67.197291,20,...,8.0,1.0,19,0.0,1.0,1,1,1.0,4.5,3
3,TCHWPBT,1.0,1.0,1.0,1.0,0,3,-96.407538,-67.197291,23,...,5.0,1.0,16,0.0,1.0,1,11,1.0,4.5,3
4,TCHWPBT,1.0,1.0,1.0,1.0,0,3,-96.407538,-67.197291,28,...,15.0,1.0,10,0.0,1.0,1,11,1.0,4.4,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,ZGFSYCZ,1.0,1.0,1.0,1.0,1,1,0.191105,-78.613494,849,...,10.0,1.0,10,0.0,1.0,1,1,0.0,4.1,3
496,ZGFSYCZ,1.0,1.0,1.0,1.0,1,1,0.191105,-78.613494,855,...,8.0,1.0,10,0.0,1.0,1,1,0.0,4.2,3
497,ZGFSYCZ,1.0,1.0,1.0,1.0,1,1,0.191105,-78.613494,856,...,7.0,1.0,10,0.0,1.0,1,11,0.0,4.3,3
498,ZGFSYCZ,1.0,1.0,1.0,1.0,1,1,0.191105,-78.613494,858,...,3.0,1.0,10,0.0,1.0,1,11,0.0,4.2,3


In [492]:
%%time
#добавляю колонку как в SampleSubmission, которая нужна для сдачи решения
train = train.assign(CID_X_LOC_NUM_X_VENDOR = train.customer_id.astype(str) + ' X ' + 
                   train.location_number.astype(str) + ' X ' + train.id.astype(str))

test = test.assign(CID_X_LOC_NUM_X_VENDOR = test.customer_id.astype(str) + ' ' + 'X' + ' ' + 
                   test.location_number.astype(str) + ' ' + 'X'+ ' ' + test.id.astype(str))

test.head()

CPU times: user 13 s, sys: 3.12 s, total: 16.2 s
Wall time: 17.7 s


,customer_id,gender,status_x,verified_x,language_x,location_number,location_type,latitude_x,longitude_x,id,...,is_open,prepration_time,discount_percentage,status_y,verified_y,rank,language_y,vendor_rating,device_type,CID_X_LOC_NUM_X_VENDOR
0,ICE2DJP,1.0,1.0,1.0,1.0,0,0,-96.407538,-67.197291,4,...,1.0,15,0.0,1.0,1,11,1.0,4.4,3,ICE2DJP X 0 X 4
1,ICE2DJP,1.0,1.0,1.0,1.0,0,0,-96.407538,-67.197291,13,...,1.0,14,0.0,1.0,1,11,1.0,4.7,3,ICE2DJP X 0 X 13
2,ICE2DJP,1.0,1.0,1.0,1.0,0,0,-96.407538,-67.197291,20,...,1.0,19,0.0,1.0,1,1,1.0,4.5,3,ICE2DJP X 0 X 20
3,ICE2DJP,1.0,1.0,1.0,1.0,0,0,-96.407538,-67.197291,23,...,1.0,16,0.0,1.0,1,11,1.0,4.5,3,ICE2DJP X 0 X 23
4,ICE2DJP,1.0,1.0,1.0,1.0,0,0,-96.407538,-67.197291,28,...,1.0,10,0.0,1.0,1,11,1.0,4.4,3,ICE2DJP X 0 X 28


In [493]:
train.shape

(5950300, 26)

Здесь я хочу вывести новый признак - расстояние между рестораном и клиентом, но мне пока не хватило времени. Расстояние я посчитал, но вижу, как и было указано  в задании, много ошибок по координатах, в результате есть расстояние по 10000км, наверное надо это перебрать и исключить строки с такими ошибками, так как я думаю что рассточние один из ключевых признаков, что здесь есть. 

In [494]:
train[['latitude_x', 'longitude_x', 'latitude_y', 'longitude_y']]

,latitude_x,longitude_x,latitude_y,longitude_y
0,-96.407538,-67.197291,-0.588596,0.754434
1,-96.407538,-67.197291,-0.471654,0.744470
2,-96.407538,-67.197291,-0.407527,0.643681
3,-96.407538,-67.197291,-0.585385,0.753811
4,-96.407538,-67.197291,0.480602,0.552850
...,...,...,...,...
5950295,-0.105119,0.078487,-1.588060,-0.066441
5950296,-0.105119,0.078487,2.145206,0.745025
5950297,-0.105119,0.078487,0.251469,0.483632
5950298,-0.105119,0.078487,0.019817,0.587087


In [503]:
def haversine(lat1, lon1, lat2, lon2):


    # convert decimal degrees to radians
    lon1, lat1, lon2, lat2 = map(np.radians, (lon1, lat1, lon2, lat2))

    # haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = np.sin(dlat / 2) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2) ** 2
    c = 2 * np.arcsin(np.sqrt(a))
    km = 6367 * c
    return km

In [506]:
t = haversine(train.latitude_x, train.longitude_x, train.latitude_y, train.longitude_y, )
t

0          10203.015466
1          10216.055495
2          10224.302060
3          10203.377593
4          10323.510124
               ...     
5950295      165.576789
5950296      260.801879
5950297       59.976410
5950298       58.198492
5950299     1502.998503
Length: 5950300, dtype: float64

In [510]:
np.max(t)

16729.44338433654

Теперь я хочу добавить в orders колонку target, и присвоит всем ячейкам значение 1, затем объеденяю train и orders по столбцу CID_X_LOC_NUM_X_VENDOR, там где заказы был, target остается 1, там где заказа не было - меняю значение NaN на ноль, получаем бинарный целевой признак. 
Из orders я хочу взять только информацию для target, мне это кажется логичным, так как в контрольной тестовой выборке информации о заказах нет

In [445]:
orders = orders.assign(target=1)
orders.rename(columns={'CID X LOC_NUM X VENDOR':'CID_X_LOC_NUM_X_VENDOR'}, inplace = True)
orders.head()

,akeed_order_id,customer_id,item_count,grand_total,payment_mode,promo_code,vendor_discount_amount,promo_code_discount_percentage,is_favorite,is_rated,...,ready_for_pickup_time,picked_up_time,delivered_time,delivery_date,vendor_id,created_at,LOCATION_NUMBER,LOCATION_TYPE,CID_X_LOC_NUM_X_VENDOR,target
0,163238.0,92PEE24,1.0,7.6,2,NaN,0.0,NaN,NaN,No,...,NaN,NaN,NaN,2019-07-31 05:30:00,105,2019-08-01 05:30:16,0,NaN,92PEE24 X 0 X 105,1
1,163240.0,QS68UD8,1.0,8.7,1,NaN,0.0,NaN,NaN,No,...,NaN,NaN,NaN,2019-07-31 05:30:00,294,2019-08-01 05:31:10,0,Work,QS68UD8 X 0 X 294,1
2,163241.0,MB7VY5F,2.0,14.4,1,NaN,0.0,NaN,NaN,No,...,NaN,NaN,NaN,2019-07-31 05:30:00,83,2019-08-01 05:31:33,0,NaN,MB7VY5F X 0 X 83,1
3,163244.0,KDJ951Y,1.0,7.1,1,NaN,0.0,NaN,NaN,No,...,NaN,NaN,NaN,2019-07-31 05:30:00,90,2019-08-01 05:34:54,0,Home,KDJ951Y X 0 X 90,1
4,163245.0,BAL0RVT,4.0,27.2,1,NaN,0.0,NaN,NaN,No,...,NaN,NaN,NaN,2019-07-31 05:30:00,83,2019-08-01 05:35:51,0,Work,BAL0RVT X 0 X 83,1


In [446]:
%%time
train = train.merge(orders[['CID_X_LOC_NUM_X_VENDOR', 'target']], on = ['CID_X_LOC_NUM_X_VENDOR'], how='left')
print (train.shape)
print (train.target.shape)
train.head(200)


(6005461, 27)
(6005461,)
CPU times: user 7.73 s, sys: 5.57 s, total: 13.3 s
Wall time: 15.2 s


,customer_id,gender,status_x,verified_x,language_x,location_number,location_type,latitude_x,longitude_x,id,...,prepration_time,discount_percentage,status_y,verified_y,rank,language_y,vendor_rating,device_type,CID_X_LOC_NUM_X_VENDOR,target
0,TCHWPBT,1.0,1.0,1.0,1.0,0,3,-96.407538,-67.197291,4,...,15,0.0,1.0,1,11,1.0,4.4,3,TCHWPBT X 0 X 4,NaN
1,TCHWPBT,1.0,1.0,1.0,1.0,0,3,-96.407538,-67.197291,13,...,14,0.0,1.0,1,11,1.0,4.7,3,TCHWPBT X 0 X 13,NaN
2,TCHWPBT,1.0,1.0,1.0,1.0,0,3,-96.407538,-67.197291,20,...,19,0.0,1.0,1,1,1.0,4.5,3,TCHWPBT X 0 X 20,NaN
3,TCHWPBT,1.0,1.0,1.0,1.0,0,3,-96.407538,-67.197291,23,...,16,0.0,1.0,1,11,1.0,4.5,3,TCHWPBT X 0 X 23,NaN
4,TCHWPBT,1.0,1.0,1.0,1.0,0,3,-96.407538,-67.197291,28,...,10,0.0,1.0,1,11,1.0,4.4,3,TCHWPBT X 0 X 28,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,TCHWPBT,1.0,1.0,1.0,1.0,1,1,-96.645619,-67.169154,849,...,10,0.0,1.0,1,1,0.0,4.1,3,TCHWPBT X 1 X 849,NaN
196,TCHWPBT,1.0,1.0,1.0,1.0,1,1,-96.645619,-67.169154,855,...,10,0.0,1.0,1,1,0.0,4.2,3,TCHWPBT X 1 X 855,NaN
197,TCHWPBT,1.0,1.0,1.0,1.0,1,1,-96.645619,-67.169154,856,...,10,0.0,1.0,1,11,0.0,4.3,3,TCHWPBT X 1 X 856,NaN
198,TCHWPBT,1.0,1.0,1.0,1.0,1,1,-96.645619,-67.169154,858,...,10,0.0,1.0,1,11,0.0,4.2,3,TCHWPBT X 1 X 858,NaN


In [447]:
train['target'].fillna(0, inplace = True)

Тут дальше просто попытка построить хоть какую-то модель, к сожалению, она пока неудачная(( 

In [449]:
f = ['gender', 'status_x', 'verified_x', 'language_x', 'location_number', 'location_type', 'vendor_category_en',
    'vendor_category_id', 'delivery_charge', 'serving_distance', 'is_open', 'prepration_time',
    'discount_percentage', 'status_y', 'verified_y', 'rank', 'language_y', 'vendor_rating', 'device_type']

In [450]:
X = train[f]
y = train['target']
X.isnull().sum(axis = 0)

gender                 149288
status_x               149288
verified_x             149288
language_x             149288
location_number             0
location_type               0
vendor_category_en          0
vendor_category_id          0
delivery_charge             0
serving_distance            0
is_open                     0
prepration_time             0
discount_percentage         0
status_y                    0
verified_y                  0
rank                        0
language_y                  0
vendor_rating               0
device_type                 0
dtype: int64

In [452]:
X = X.loc[0:5500000].values
y = y.loc[0:5500000].values

In [453]:
np.sum(y)

129554.0

In [454]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size = 0.3)

In [455]:
model2 = xgb.XGBClassifier()
model2.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [456]:
y_pred = model2.predict(X_test)

In [457]:
f1_score = metrics.f1_score(y_test, y_pred)
f1_score

0.0

In [458]:
np.sum(y_pred)

0.0